## importing libraries

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
tf.__version__

'2.18.0'

# Part1: Data preprocessing

### Processing the training set

In [8]:
traind_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip=True)

In [9]:
training_set = traind_datagen.flow_from_directory(
    'training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Processing the test set

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
test_set = test_datagen.flow_from_directory(
    'test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


# Part2: Building CNN

### Initialising the CNN

In [18]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [20]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3 , activation="relu", input_shape=(64, 64, 3)))

### Step 2 - Pooling

In [21]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size =2, strides=2))

### Adding 2nd layer

In [22]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3 , activation="relu"))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size =2, strides=2))

### Step 3 - Flattening

In [23]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full connection

In [24]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output layer

In [26]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part3: Training CNN

### Compiling CNN

In [29]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training CNN

In [30]:
cnn.fit(x = training_set, validation_data=test_set, epochs = 25)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 606ms/step - accuracy: 0.5266 - loss: 0.6938

C:\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 174s 689ms/step - accuracy: 0.5266 - loss: 0.6937 - val_accuracy: 0.5960 - val_loss: 0.6805
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 63s 253ms/step - accuracy: 0.6003 - loss: 0.6644 - val_accuracy: 0.5915 - val_loss: 0.6676
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 72s 288ms/step - accuracy: 0.6696 - loss: 0.6069 - val_accuracy: 0.7115 - val_loss: 0.5773
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 68s 272ms/step - accuracy: 0.7103 - loss: 0.5614 - val_accuracy: 0.7350 - val_loss: 0.5350
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 56s 222ms/step - accuracy: 0.7446 - loss: 0.5222 - val_accuracy: 0.7590 - val_loss: 0.5065
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 62s 247ms/step - accuracy: 0.7600 - loss: 0.4845 - val_accuracy: 0.7695 - val_loss: 0.4962
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 79s 315ms/step - accuracy: 0.7855 - loss: 0.4609 - val_accuracy: 0.7815 - val_loss: 0.4618
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 67s 266ms/step - accuracy: 0.7939 - loss: 0.4363 - va

# Part4: Making single prediction

In [41]:
import numpy as np
from keras.preprocessing import image

In [45]:
def predict(path):
    # Load the image
    test_image = image.load_img(path, target_size=(64, 64))
    
    # Convert the image to a numpy array
    img_array = image.img_to_array(test_image)
    
    # Normalize the image
    img_array = img_array / 255.0
    
    # Add a batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make a prediction
    result = cnn.predict(img_array)
    
    # Get the class indices
    class_indices = training_set.class_indices
    
    # Determine the prediction
    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'
    
    return prediction

In [46]:
# Example usage
print(predict('single_prediction/cat_or_dog_1.jpg'))  # This is a dog image
print(predict('single_prediction/cat_or_dog_2.jpg'))  # This is a cat image

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
cat


## Saving model

In [47]:
cnn.save('../model/cat_or_dog.keras')